# Kaggle Neuroblastoma Detection
## Lachlan Dryburgh 2021

Tensorflow implementation of a u-net image segmentation convolutional nerual network.  Trained to label neurons, astrocytes and neuroglioblastoma cell in microscope images.

## Imports and Defines

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
from skimage import morphology
from skimage import filters



IMG_HEIGHT = 520
IMG_WIDTH = 704
NUM_CLASS = 3

SEED = 351

CELL = {
    'shsy5y':0,
    'astro':1,
    'cort':2,
    's':0,
    'a':1,
    'c':2
}



In [2]:
def set_strategy():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    except ValueError:
        tpu = None
        gpus = tf.config.experimental.list_logical_devices("GPU")

    if tpu:
        strategy = tf.distribute.TPUStrategy(tpu)
        print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    elif len(gpus) > 1:
        strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
        print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
    elif len(gpus) == 1:
        strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
        print('Running on single GPU ', gpus[0].name)
    else:
        strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
        print('Running on CPU')

    print("Number of accelerators: ", strategy.num_replicas_in_sync)

    return strategy

strategy = set_strategy()

## Importing Images

In [3]:
train_img = "../input/sartorius-cell-instance-segmentation/train"
train_csv = "../input/sartorius-cell-instance-segmentation/train.csv"
test_img = "../input/sartorius-cell-instance-segmentation/test"
semi_supervised = "../input/sartorius-cell-instance-segmentation/train_semi_supervised/"

df = pd.read_csv(train_csv)
df.head()


In [4]:
ids = df['id'].unique()
len(ids)

In [5]:
df.groupby('cell_type').size()

## Image pixel annotation mask 

In [6]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)
    

def generate_mask(id, df, shape):
    mask = np.zeros((shape[0], shape[1], shape[2]), dtype=np.uint8)
    
    for index, row in df[df['id']==id].iterrows():
        
        c = CELL[row['cell_type']]
        
        m = rle_decode(row['annotation'], (IMG_HEIGHT, IMG_WIDTH))
        #m = filters.rank.minimum(m, np.array([[1,1,1],[1,1,1],[1,1,1]]))
        
        mask[:,:,c] += np.array(m, dtype=np.uint8)
        mask = mask.clip(0,1)
       
    return mask



In [7]:
import random

random.seed(SEED)

In [8]:
random_id = random.choice(ids)

sample_path = f"{train_img}/{random_id}.png"

im = plt.imread(sample_path)
m = generate_mask(random_id, df, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS))

print(random_id)

figure, ax = plt.subplots(1,2,figsize=(15,7))
plt.suptitle(random_id,fontweight="bold", size=20)
ax[0].imshow(im, cmap = 'seismic')
ax[1].imshow(np.array(m, dtype=np.float32))

In [9]:
def mask_edges(id, df, shape):
    mask = np.zeros(shape, dtype=np.uint8)
    
    for index, row in df[df['id']==id].iterrows():
                
        m = rle_decode(row['annotation'], shape)
        
        
        m = m - filters.rank.minimum(m, np.array([[1,1,1],[1,1,1],[1,1,1]]))
        
        mask += np.array(m, dtype=np.uint8)
        mask = mask.clip(0,1)
    
    return mask

def generate_output(id, df):
    out = np.zeros((IMG_HEIGHT, IMG_WIDTH, 4), dtype=np.uint8)
    
    e = mask_edges(id, df,(IMG_HEIGHT, IMG_WIDTH))
    m = generate_mask(id, df, (IMG_HEIGHT, IMG_WIDTH, 3))
    
    out[:,:,0] = e
    out[:,:,1:4] = m
    
    return out

In [10]:
out = generate_output(random_id, df)

plt.imshow(out[:,:,0])

In [11]:
im = im.reshape((IMG_HEIGHT, IMG_WIDTH, 1))

np.shape(im)

In [12]:
shsy5y = '1c4f14cce8ee'
astro = '129f894abe35'
cort = '95de75855f80'

s_path = f"{train_img}/{shsy5y}.png"
a_path = f"{train_img}/{astro}.png"
c_path = f"{train_img}/{cort}.png"

s_im = plt.imread(s_path)
a_im = plt.imread(a_path)
c_im = plt.imread(c_path)

figure, ax = plt.subplots(2,3,figsize=(15,9))
plt.suptitle("Images and Masks",fontweight="bold", size=20)

ax[0,0].imshow(s_im, cmap = 'seismic')
ax[0,0].set_title(f"SH-SY5Y  - {shsy5y}")
ax[1,0].imshow(np.array(generate_mask(shsy5y, df, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS)), dtype=np.float32))
ax[1,0].set_title(f"{shsy5y} annotation mask")
ax[0,1].imshow(a_im, cmap = 'seismic')
ax[0,1].set_title(f"Astrocyte - {astro}")
ax[1,1].imshow(np.array(generate_mask(astro, df, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS)), dtype=np.float32))
ax[1,1].set_title(f"{astro} annotation mask")
ax[0,2].imshow(c_im, cmap = 'seismic')
ax[0,2].set_title(f"Coritical Neuron  - {cort}")
ax[1,2].imshow(np.array(generate_mask(cort, df, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS)), dtype=np.float32))
ax[1,2].set_title(f"{cort} annotation mask")
[axi.set_axis_off() for axi in ax.ravel()]
figure.tight_layout()
figure.show()


## Split the data into training and validation sets

In [13]:
from sklearn.model_selection import train_test_split

train_ids, valid_ids = train_test_split(ids, test_size=0.2, random_state=SEED)
print(len(train_ids), len(valid_ids))

## Generate Training dataset

In [14]:
from os import walk

filenames = next(walk(semi_supervised), (None, None, []))[2] 

In [15]:
import shutil

try:
    os.mkdir("../semi_supervised")
except:
    print("Already exists")

try:
    os.mkdir("../semi_supervised/a")
except:
    print("Already exists")
    
try:    
    os.mkdir("../semi_supervised/c")
except:
    print("Already exists")
    
try:
    os.mkdir("../semi_supervised/s")
except:
    print("Already exists")

for f in filenames:
    shutil.copyfile(f"{semi_supervised}{f}", f"../semi_supervised/{f[0]}/{f}")

In [16]:
class_training = tf.keras.utils.image_dataset_from_directory(
    "../semi_supervised/",
    color_mode = 'grayscale',
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32)

class_validation = tf.keras.utils.image_dataset_from_directory(
    "../semi_supervised/",
    color_mode = 'grayscale',
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32)

## Create datasets for supervised learning
Using a generator allows data to be loaded into memory in chucks rather than all at once.

In [17]:
def train_generator(ids, df):
    
    for image_id in ids:
        img = plt.imread(os.path.join(train_img, image_id) + '.png') 
        img = img.reshape((IMG_HEIGHT,IMG_WIDTH, 1))
        
        #mask = generate_output(image_id, df)
        mask = generate_mask(image_id, df, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS))
               
        yield img, mask
        
def edge_generator(ids, df):
    
    for image_id in ids:
        img = plt.imread(os.path.join(train_img, image_id) + '.png') 
        img = img.reshape((IMG_HEIGHT,IMG_WIDTH, 1))
        
        mask = mask_edges(image_id, df,(IMG_HEIGHT,IMG_WIDTH))
        
        mask = mask.reshape((IMG_HEIGHT,IMG_WIDTH, 1))
               
        yield img, mask

In [18]:
train_ds = tf.data.Dataset.from_generator(
    lambda : train_generator(train_ids, df),
    output_types=(tf.float32, tf.int32),
    output_shapes=((IMG_HEIGHT, IMG_WIDTH, 1), (IMG_HEIGHT, IMG_WIDTH, 3)))

    

valid_ds = tf.data.Dataset.from_generator(
    lambda : train_generator(valid_ids, df),
    output_types=(tf.float32, tf.int32),
    output_shapes=((IMG_HEIGHT, IMG_WIDTH, 1), (IMG_HEIGHT, IMG_WIDTH, 3)))

print(train_ds)
print(valid_ds)
    

In [19]:
DROPUOUT_RATE = 0.1

cf = 16

## Define  model
We are actually defining 2 models.

The downstack of the u-net is used as image classifier so that it can be trained where we only have labels for the enire image rather than pixels.  This will allow transfer learning.

The Downstack feeds back into the upstack for our unet pixel classifier.


In [20]:
with strategy.scope():
    in1 = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH,1))

    conv1 = layers.Conv2D(cf, (7, 7), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    drop1 = layers.Dropout(DROPUOUT_RATE)(conv1)
    conv2 = layers.Conv2D(cf, (7, 7), activation='relu', kernel_initializer='he_normal', padding='same')(drop1)
    pool1 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(cf*2, (5, 5), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    drop2 = layers.Dropout(DROPUOUT_RATE)(conv3)
    conv4 = layers.Conv2D(cf*2, (5, 5), activation='relu', kernel_initializer='he_normal', padding='same')(drop2)
    pool2 = layers.MaxPooling2D((2, 2))(conv4)

    conv5 = layers.Conv2D(cf*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    drop3 = layers.Dropout(DROPUOUT_RATE)(conv5)
    conv6 = layers.Conv2D(cf*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(drop3)
    pool3 = layers.MaxPooling2D((2, 2))(conv6)

    conv7 = layers.Conv2D(cf*8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    drop4 = layers.Dropout(DROPUOUT_RATE)(conv7)
    conv8 = layers.Conv2D(cf*8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(drop4)

    up1 = layers.concatenate([layers.Conv2DTranspose(cf*4, (2, 2), strides=(2, 2), padding='same')(conv8), conv6], axis=-1)
    conv9 = layers.Conv2D(cf*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
    drop5 = layers.Dropout(DROPUOUT_RATE)(conv9)
    conv10 = layers.Conv2D(cf*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9)

    up2 = layers.concatenate([layers.Conv2DTranspose(cf*2, (2, 2), strides=(2, 2), padding='same')(conv10), conv4], axis=-1)
    conv11 = layers.Conv2D(cf*2, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    drop6 = layers.Dropout(DROPUOUT_RATE)(conv11)
    conv12 = layers.Conv2D(cf*2, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(drop6)

    up3 = layers.concatenate([layers.Conv2DTranspose(cf, (2, 2), strides=(2, 2), padding='same')(conv12), conv2], axis=-1)
    conv13 = layers.Conv2D(cf, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up3)
    drop7 = layers.Dropout(DROPUOUT_RATE)(conv13)
    conv14 = layers.Conv2D(cf, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(drop7)
    segmentation = layers.Conv2D(3, (1, 1), activation='sigmoid', name='seg')(conv14)
    

    flat1 = layers.Flatten()(conv8)
    dense1 = layers.Dense(256, activation = 'relu')(flat1)
    dense2 = layers.Dense(256, activation = 'relu')(dense1)
    dense3 = layers.Dense(64, activation = 'relu')(dense2)
    class_box = layers.Dense(4, activation = 'softmax')(dense3)

    class_model = keras.Model(inputs=[in1], outputs=[class_box])

    model = keras.Model(inputs=[in1], outputs=[segmentation])


    class_model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['acc'])

    model.compile(optimizer="adam", loss = {'seg': 'binary_crossentropy'}, metrics={'seg': ['acc']})

## Compile the class-box model


In [21]:
keras.utils.plot_model(class_model, "downstack.png", show_shapes=True)

## Compile the full model

In [22]:
keras.utils.plot_model(model, "full_model.png", show_shapes=True)

## Train the downstack

In [23]:
AUTOTUNE=tf.data.AUTOTUNE

class_t_ds = class_training.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
class_v_ds = class_validation.cache().prefetch(buffer_size=AUTOTUNE)

In [24]:
EPOCHS =50

#his = class_model.fit(class_t_ds,
#                epochs=EPOCHS,
#                validation_data=class_v_ds
#)

In [25]:
#plt.plot(his.history['loss'][1:])
#plt.plot(his.history['val_loss'][1:])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'val'], loc='upper left')
#plt.show()

In [ ]:
#plt.plot(his.history['acc'])
#plt.plot(his.history['val_acc'])
#plt.title('model loss')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'val'], loc='upper left')
#plt.show()

## Freeze the Downstack

In [ ]:
freeze =False

conv1.trainable = not freeze 
conv2.trainable = not freeze 
conv3.trainable = not freeze 
conv4.trainable = not freeze 
conv5.trainable = not freeze 
conv6.trainable = not freeze 
conv7.trainable = not freeze 
conv8.trainable = not freeze 


pool1.trainable = not freeze 
pool2.trainable = not freeze 
pool3.trainable = not freeze 

In [26]:
class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=SEED):
        super().__init__()
        
        self.augment_inputs = tf.keras.layers.RandomFlip('horizontal_and_vertical', seed=seed)
        self.augment_labels = tf.keras.layers.RandomFlip('horizontal_and_vertical', seed=seed)
        
    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)
        return inputs, labels

In [27]:
BATCH_SIZE = 16
BUFFER_SIZE = 32

EPOCHS = 200

TRAIN_LENGTH = len(train_ids)
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

VALID_LENGTH = len(valid_ids)
VALIDATION_STEPS = VALID_LENGTH // BATCH_SIZE

In [28]:
t_ds = (
    train_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    .map(Augment())
    .prefetch(AUTOTUNE))

v_ds = (
    valid_ds
    .batch(BATCH_SIZE)
    .repeat()
    .prefetch(AUTOTUNE))

In [29]:
model_history = model.fit(t_ds, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=v_ds)

In [30]:
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [31]:
plt.plot(model_history.history['acc'])
plt.plot(model_history.history['val_acc'])
plt.title('model loss')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [42]:
vis_img = random.choice(valid_ids)
im = plt.imread(os.path.join(train_img, vis_img) + '.png')
im = im.reshape((IMG_HEIGHT, IMG_WIDTH, 1))

pred = model.predict(np.array([im]))

pred.shape

p = pred[0,:,:,:]
p = p.reshape(IMG_HEIGHT, IMG_WIDTH,3)



pm = np.round(p,0)
pm = np.dot(pm, [1,1,1])

m = generate_mask(vis_img, df, (IMG_HEIGHT, IMG_WIDTH, NUM_CLASS))

figure, ax = plt.subplots(1,4,figsize=(18,7))
plt.suptitle(vis_img,fontweight="bold", size=20)
ax[0].set_title("Image")
ax[0].imshow(im, cmap = 'seismic')
ax[1].set_title("Annotation")
ax[1].imshow(np.array(m, dtype=np.float32))
ax[2].set_title("Prediction Probability")
ax[2].imshow(p)
ax[3].set_title("Mask Prediction")
ax[3].imshow(pm)
[axi.set_axis_off() for axi in ax.ravel()]
figure.show()


In [ ]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    ref: https://www.kaggle.com/dragonzhang/positive-score-with-detectron-3-3-inference
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [40]:
from skimage import measure

l = measure.label(pm)

num_lab = np.max(l)

In [41]:
for i in range(num_lab):
    